# 2d Bin Packing Problem

The 2d bin-packing problem is a staple of optimization research.

Given bins of integer size $(h_b \times w_b)$, and a set of $k$ objects of (potentially) different integer sizes $((h_1, w_1), ..., (h_k, w_k))$, find the smallest number of bins $b$ that all objects can be packed into and assign objects to bins and positions so that they do not overlap: $((b_1, y_1, x_1), ..., (b_k, y_k, x_k))$.

(There are generally multiple valid assignments, and we ignore rotations for now. Both algorithms we discuss can be extended to support rotations.)

In this paper we present a novel method to represent this problem as an integer program that reduces the number of constraints needed by an order of magnitude and greatly improves solver speed. Experiments timing the the two methods are posted [here](https://docs.google.com/spreadsheets/d/1nOmQeqqyNGrBDv8WexNodaEqDQn7Dklin0k3WtaR3bA/edit?usp=sharing).


## Positions and Covering (P&C)

[_Positions and Covering_ (Cid-Garcia and Rios-Solis, 2020)](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0229358) is the most recent solution proposed, and it solves the problem quite elegantly. Instead of explaining it mechanically as in the original paper we describe it as a sequence of transformations on the problem representation. The goal, then, is to find a sequence of transformations that (1) takes as few additional variables and constraints as possible, and (2) allows us to represent the various goal constraints as efficiently as possible.

Positions and covering splits the entire problem into a sequence of two transformations, the _positions_ stage and the _coverings_ stage. They find the number of bins $b$ by trying increasing numbers until an integer program can be solved.

We begin with the key decision variables $((b_1, y_1, x_1), ..., (b_k, y_k, x_k))$. Object $i$ is placed in bin $b_i \in [1, b]$, occupying the intersection of rows $[y_i, y_i + h_i - 1]$ and columns $[x_i, x_i + w_i - 1]$. Here we consider the representation of all objects:

$$
\underbrace{
\begin{bmatrix}
b_1 & y_1 & x_1\\
b_2 & y_2 & x_2\\
&...\\
b_k & y_k & x_k\\
\end{bmatrix}
}_{\text{decision variables}}
$$

They first represent these as the __positions__ stage, where the representation of object $i$ is encoded as a $b \times h_b \times w_b$ matrix corresponding to position assignment. Exactly one valid position must hold value 1, and the remaining must be 0. Object $i$ is represented as:

$$
\underbrace{
\begin{bmatrix}
((b_i = 1) \land (y_i = 1) \land (x_i = 1)) & ((b_i = 1) \land (y_i = 1) \land (x_i = 2)) & ... \\
((b_i = 1) \land (y_i = 2) \land (x_i = 1)) & ...\\
...& & ((b_i = 1) \land (y_i = b_h) \land (x_i = b_w))\\
\end{bmatrix}
}_{\text{$b$ such bins for each object}}
$$

(Note that this shows the logical conditions of each variable, not actually how it is implemented. Both P&C and H&C sidestep this encoding/decoding process.)

The total number of variables in the positions stage is $\mathcal O(k * b * b_h * b_w)$.

To add non-overlapping constraints they then use the _coverings_ representation, where they encode the cells that an object covers conditioned on the positions matrix.

We construct a coverings matrix also of size $[b, b_h, b_w]$ where the cell at position $(b_i, y_i, x_i)$ is 1 if object $i$ is in any position within bin $b_i$ that overlaps with the cell $(y_i, x_i)$.

(Another equivalent and perhaps more intuitive way to say this is is that if the positions matrix is 1 at $(b_i, y_i, x_i)$, then a rectangular region in the covering matrix, starting from $(b_i, y_i, x_i)$ and extending $h_i$ rows and $w_i$ columns, must also be 1.)

$$
\underbrace{
\begin{bmatrix}
0 & 0 & 0 & 0 & 0\\
0 & 1 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0\\
\end{bmatrix}
}_{\text{positions}}
\rArr
\underbrace{
\begin{bmatrix}
0 & 0 & 0 & 0 & 0\\
0 & 1 & 1 & 1 & 0\\
0 & 1 & 1 & 1 & 0\\
0 & 1 & 1 & 1 & 0\\
0 & 0 & 0 & 0 & 0\\
\end{bmatrix}
}_{\text{covering for a $3\times 3$ object}}
$$

We observe that constructing the coverings matrix is a very expensive operation, adding $\mathcal O(k * b * b_h * b_w * \max(h_\circ) * \max(w_\circ)) \subset \mathcal O(k * b * b_h^2 * b_w^2)$ constraints. This is despite the fact that the number of variables is similar to the positions stage at $\mathcal O(k * b * b_h * b_w)$.

After all this work, we can now very simply add the non-overlapping constraint by setting the sum of all values across all objects to be at most 1 in each position in each bin. We can conceptualize this as an "overlap matrix" of size $b * b_h * b_w$, which stores the number of objects that occupy each position in each bin. Constraining all values in this matrix to be at most 1 effectively represents the non-overlapping constraints and allows us to solve 2d bin-packing problems.


## Hough and Cover (H&C)

The _positions_ stage of P&C belongs to a class of techniques called _parameter-space transforms_ that are routinely rediscovered across a range of disciplines. I first learned about these as the _Hough transform_ -- a mechanism to identify patterns in images patented in the 1960s that was a cornerstone of classic computer vision.

The H&C technique uses exactly the same _positions_ stage as P&C. Our contribution lies entirely in making the computation of the covering stage more efficient. We begin by observing that the covering matrix in P&C can be represented as a running sum of a sparse pre-image matrix. We call that the _delta_ matrix.

$$
\text{RunningSum}\left(
\underbrace{
\begin{bmatrix}
 1 & 0 & 0 & -1 & 0\\
 0 & 0 & 0 &  0 & 0\\
 0 & 0 & 0 &  0 & 0\\
-1 & 0 & 0 &  1 & 0\\
 0 & 0 & 0 &  0 & 0\\
\end{bmatrix}
}_{\text{delta matrix}}\right)
=
\underbrace{
\begin{bmatrix}
1 & 1 & 1 & 0 & 0 \\
1 & 1 & 1 & 0 & 0 \\
1 & 1 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
\end{bmatrix}
}_{\text{covering matrix}}
$$

For clarity: the running sum operation is the same as the cumulative sum operation in [`numpy`](https://numpy.org/doc/stable/reference/generated/numpy.cumsum.html), [`pandas`](https://pandas.pydata.org/docs/reference/api/pandas.Series.cumsum.html), [`Matlab`](https://www.mathworks.com/help/matlab/ref/cumsum.html), etc. If $A \gets \text{RunningSum}(B)$, then $A(i, j) = \text{sum}(B(1:i, 1:j))$.

$$
\text{RunningSum}\left(
\underbrace{
\begin{bmatrix}
 1 & 0 & 0 & -1 & 0\\
 0 & 0 & 0 &  0 & 0\\
 0 & 0 & 0 &  0 & 0\\
-1 & 0 & 0 &  1 & 0\\
 0 & 0 & 0 &  0 & 0\\
\end{bmatrix}
}_{\text{delta matrix $D$}}\right)
=
\underbrace{
\begin{bmatrix}
1 & 1 & 1 & 0 & 0 \\
1 & 1 & 1 & 0 & 0 \\
1 & 1 & 1 & 0 & 0 \\
0 & 0 & ? & ? & ? \\
? & ? & ? & ? & ? \\
\end{bmatrix}
}_{\text{covering matrix $A$}}
$$

We can efficiently express the running sum transformation by building the representation in row-major order. At the next cell to be computed $A(i,j)$ in the above matrix, we can write the constraint as $A(i,j) \gets D(i,j) + A(i,j-1) + A(i-1,j) - A(i-1, j-1)$.

Now, for the true trick behind H&C. We observe that the running sum operation is a linear operation. Our conceptual overlap matrix (which counts the number of objects occupying each position in each bin) can be written as the sum over the RunningSum of each delta matrix, or equivalently as the RunningSum over the sum of each delta matrix. This switch in order allows us to incur the overhead of the RunningSum operation exactly once!

Counting the constraints needed to encode this: the delta matrix incurs a constant number of constraints per element of the positions matrix for a total of $\mathcal O(k * b * b_h * b_w)$ constraints. The summation step applies to each variable exactly once to incur $\mathcal O(k * b * b_h * b_w)$ constraints, and the running-sum step touches each element in the overlap matrix a fixed number of times, leading to $\mathcal O(b * b_h * b_w)$. The dominating term is $\mathcal O(k * b * b_h * b_w)$ constraints, which is a tremendous improvement over P&C.


## Experiments

We run some initial experiments comparing (our implementation of) P&C and H&C on some reference problems from [Unibo](http://or.dei.unibo.it/general-files/best-known-solution-and-lower-bound-each-instance). We implement both in Julia using Gurobi as the underlying solver.

Initial results, posted [here](https://docs.google.com/spreadsheets/d/1nOmQeqqyNGrBDv8WexNodaEqDQn7Dklin0k3WtaR3bA/edit?usp=sharing), are promising. H&C is faster then P&C on most problems typically with a 3x speedup, but in some rare cases we can sometimes take longer than P&C.

More experiments (and not on a laptop!) need to be conducted to be sure.


### Asides

The running-sum and delta operations are, together, essentially equivalent to convolving the positions matrix of each object with a rectangle of ones in its size. This also gives us a clue that the fundamental operation is linear, and we can re-order operations around it.

Also, while the running-sum concept undoubtedly existed earlier, I first encountered it in the [1999 Viola-Jones face detection paper](https://www.cs.cmu.edu/~efros/courses/LBMV07/Papers/viola-cvpr-01.pdf) where it was used to evaluate cascades of Haar-like features in real-time on 1999-era hardware. In their paper, they called this technique the "integral image".



## TODO

1. We need to implement the "polyhedron strengthening" losses in both sides.
2. Explain the heuristic minimal- and maximal number of bins.
3. Maybe explore heuristics in searching for b.
4. Acknowledge that we save a lot of annoyance by encoding and decoding the positions transformation _outside_ the integer program, unlike what is proposed in the P&C paper.
5. We can make (potentially) huge gains by deduplicating objects and using the object-demand formulation.
6. Since we have a bound for the top, we can potentially discover $b$ in a single integer program by penalizing any assignment of objects to bin $m$ the amount $k^{(m - b_\text{lower})}$. This will guarantee that bins are filled in order, potentially at cost to solve time. (It may be faster for $b$ close to lower bound $b_\text{lower}$ to be solved by guess-and-check on $m$ instead of this, particularly when $b_\text{upper}$ is large.)
7. Both P&C and H&C can also be expressed as a CP rather than an IP. This may lead to faster solves, or it may not.
8. Is this the lowest-bound algorithm? I can't see how (unless we exploit some sort of structure in the problem) we can do better than $\mathcal O(k * b * b_h * b_w)$ constraints.